In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("train.csv")
dataTest = pd.read_csv("test.csv")
print(data.head())

                                            sentence  \
0              2 tablespoons vegetable oil , divided   
1                       2 tablespoons dried marjoram   
2  1 large red onion , 1/4-inch slices pulled int...   
3             2 jalapeno peppers , seeded and minced   
4                             1 tablespoon olive oil   

                                  tags  
0      QUANTITY UNIT NAME NAME O STATE  
1                QUANTITY UNIT DF NAME  
2  QUANTITY SIZE NAME NAME O O O O O O  
3   QUANTITY NAME NAME O STATE O STATE  
4              QUANTITY UNIT NAME NAME  


In [3]:
#converting the raw dataset in to token, tag pairs

formatted_sentences = []

#iterrows creates a iterator for pd series
for index, row in data.iterrows():

    tokens = row["sentence"].split()
    tags = row["tags"].split()

    sentence = [(token, tag) for token, tag in zip(tokens, tags)]

    formatted_sentences.append(sentence)

print(formatted_sentences)

formatted_sentences_test = []

#iterrows creates a iterator for pd series
for index, row in dataTest.iterrows():

    tokens = row["sentence"].split()
    tags = row["tags"].split()

    sentence = [(token, tag) for token, tag in zip(tokens, tags)]

    formatted_sentences_test.append(sentence)

print(formatted_sentences_test)


[[('2', 'QUANTITY'), ('tablespoons', 'UNIT'), ('vegetable', 'NAME'), ('oil', 'NAME'), (',', 'O'), ('divided', 'STATE')], [('2', 'QUANTITY'), ('tablespoons', 'UNIT'), ('dried', 'DF'), ('marjoram', 'NAME')], [('1', 'QUANTITY'), ('large', 'SIZE'), ('red', 'NAME'), ('onion', 'NAME'), (',', 'O'), ('1/4-inch', 'O'), ('slices', 'O'), ('pulled', 'O'), ('into', 'O'), ('rings', 'O')], [('2', 'QUANTITY'), ('jalapeno', 'NAME'), ('peppers', 'NAME'), (',', 'O'), ('seeded', 'STATE'), ('and', 'O'), ('minced', 'STATE')], [('1', 'QUANTITY'), ('tablespoon', 'UNIT'), ('olive', 'NAME'), ('oil', 'NAME')], [('extra', 'NAME'), ('virgin', 'NAME'), ('olive', 'NAME'), ('oil', 'NAME'), (',', 'O'), ('for', 'O'), ('drizzling', 'O')], [('1/3', 'QUANTITY'), ('cup', 'UNIT'), ('sliced', 'STATE'), ('pitted', 'STATE'), ('ripe', 'STATE'), ('olives', 'NAME'), ('-LRB-', 'O'), ('optional', 'O'), ('-RRB-', 'O')], [('3', 'QUANTITY'), ('cloves', 'UNIT'), ('garlic', 'NAME'), (',', 'O'), ('minced', 'STATE')], [('4', 'QUANTITY'), 

In [4]:
outputTrain = "finalTrain.tsv"
outputTest = "finalTest.tsv"

with open(outputTrain, "w") as f:
    for sentence in formatted_sentences:
        for token, tag in sentence:
            f.write(f"{token}\t{tag}\n")


with open(outputTest, "w") as f:
    for sentence in formatted_sentences_test:
        for token, tag in sentence:
            f.write(f"{token}\t{tag}\n")

        f.write("\n")

In [38]:
# !pip install transformers datasets seqeval


In [5]:
import pandas as pd

def parse_tsv(file_path):
    with open(file_path, 'r') as f:
        data = f.read().strip().split('\n\n')  # Each sentence is separated by a blank line

    sentences = []
    labels = []

    for sentence in data:
        words = []
        tags = []
        for line in sentence.strip().split('\n'):
            if line:  # Skip blank lines
                word, tag = line.split('\t')
                # words.append(word)
                # tags.append(tag)
                sentences.append(word)
                labels.append(tag)
        # sentences.append(words)
        # labels.append(tags)

    return sentences, labels

# Example usage
train_sentences, train_labels = parse_tsv('finalTrain.tsv')
test_sentences, test_labels = parse_tsv('finalTest.tsv')

In [6]:
#building up train data and test data
train_data = pd.DataFrame({
    "sentence_id": train_sentences,
    "words": train_sentences,
    "labels": train_labels
})

test_data = pd.DataFrame({
    "sentence_id": test_sentences,
    "words": test_sentences,
    "labels": test_labels
})

In [7]:
train_data.head()

sentence_id        words    labels
0            2            2  QUANTITY
1  tablespoons  tablespoons      UNIT
2    vegetable    vegetable      NAME
3          oil          oil      NAME
4            ,            ,         O

In [9]:
# !pip install simpletransformers

zsh:1: command not found: pip


In [ ]:
pip install simpletransformers

In [ ]:
pip install torch


In [14]:
from simpletransformers.ner import NERModel,NERArgs

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
label = train_data["labels"].unique().tolist()
label

['QUANTITY', 'UNIT', 'NAME', 'O', 'STATE', 'DF', 'SIZE', 'TEMP']

In [16]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32


In [ ]:
model = NERModel('bert', 'bert-base-cased',labels=label,args =args) 

ValueError: 'use_cuda' set to True when cuda is unavailable.Make sure CUDA is available or set use_cuda=False.

In [47]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [48]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_model.py:758: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 1:   0%|          | 0/74 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_model.py:782: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


(74, 0.9135152949271975)

In [49]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/47 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_model.py:1303: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: STATE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: QUANTITY seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NAME seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: TEMP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-

In [50]:
result

{'eval_loss': 0.5113360580611737,
 'precision': 0.7163876204972095,
 'recall': 0.6094087181700475,
 'f1_score': 0.6585820895522388}

In [53]:
prediction, model_output = model.predict(["4 tablespoons oyster sauce"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [54]:
prediction

[[{'4': 'O'}, {'tablespoons': 'O'}, {'oyster': 'NAME'}, {'sauce': 'NAME'}]]

In [ ]:
week 1